In [9]:
import pandas as pd
import boto3
import io

In [ ]:
ACCESS_KEY = ''
SECRET_KEY = ''


s3_client = boto3.client('s3', region_name='us-east-1', aws_access_key_id=ACCESS_KEY,
                         aws_secret_access_key=SECRET_KEY)
athena_client = boto3.client('athena', region_name='us-east-1', aws_access_key_id=ACCESS_KEY,
                         aws_secret_access_key=SECRET_KEY)

session = boto3.Session(aws_access_key_id=ACCESS_KEY,
                        aws_secret_access_key=SECRET_KEY)

In [36]:
df = pd.read_csv(r"C:\Users\hamupratik\Downloads\1d1d5bca-2ca1-415f-ba25-939be2e07541.csv", dtype = str)

In [12]:
source_db = 'devoted_health_prod'
QUERY_OUTPUT_LOCATION = "s3://zignaai-deidentified-claimsdata/query output/"
WORKGROUP_NAME = 'SelectionQueries-Production'

In [13]:
def run_query(athena_client, s3_client, query_string, source_db, QUERY_OUTPUT_LOCATION, WORKGROUP):
    
    try:
        response = athena_client.start_query_execution(
            QueryString=query_string,
            QueryExecutionContext={
                'Database': source_db
            },
            ResultConfiguration={
                'OutputLocation': QUERY_OUTPUT_LOCATION
            },
            WorkGroup=WORKGROUP
        )
        query_execution_id = response['QueryExecutionId']
        
        while True:
            query_status = athena_client.get_query_execution(QueryExecutionId=query_execution_id) 
            query_state = query_status['QueryExecution']['Status']['State'] 
            query_result_path = query_status['QueryExecution']['ResultConfiguration']['OutputLocation'] # location of query output
            if query_state in ['SUCCEEDED', 'FAILED', 'CANCELLED']:
                break

        if query_state == 'SUCCEEDED':
            try:
                bucket = query_result_path.split('//')[1].split('/')[0]
                key = '/'.join(query_result_path.split('//')[1].split('/')[1:])
                buffer = io.BytesIO(s3_client.get_object(Bucket=bucket, Key=key)['Body'].read()) 
                df = pd.read_csv(buffer, dtype=str) 
                return df
            except Exception as e:
                print("QUERY EXECUTION SUCCESSFULL BUT GOT ERROR WHILE CONVERSION TO DATAFRAME")
                print(e)
        else:
            print("Query execution failed Status: {}".format(query_state))
            return None
    except Exception as e:
            print("GOT EXCEPTION WHILE RUNNING QUERY")
            print(e)
            return None

In [15]:
query_string = """SELECT * FROM "zigna_reference_data"."reference_table_add_on_codes"
WHERE aoc_edit_type = 1"""

referance_data = run_query(athena_client, s3_client, query_string, source_db, QUERY_OUTPUT_LOCATION, WORKGROUP_NAME)

In [ ]:
referance_data_target_referance = referance_data[['add_on_code', 'primary_code', 'aoc_edit_effdt']].drop_duplicates()

# theres correction in aoc_edit_effdt interpretation change it later
referance_data_target_referance['aoc_edit_effdt'] = referance_data_target_referance['aoc_edit_effdt'].apply(lambda x: pd.to_datetime(str(x), format='%Y%j').strftime('%Y-%m-%d'))

In [27]:
overlapping_codes = set(referance_data_target_referance['add_on_code']).intersection(set(referance_data_target_referance['primary_code']))

referance_data_target_referance = referance_data_target_referance[~referance_data_target_referance['add_on_code'].isin(overlapping_codes)]
referance_data_target_referance = referance_data_target_referance[~referance_data_target_referance['primary_code'].isin(overlapping_codes)]


In [37]:
df['plan_paid_amount'] = df['plan_paid_amount'].astype(float)

In [38]:
add_on_map = referance_data_target_referance.groupby('add_on_code').apply(lambda x: set(x['primary_code']))
add_on_map = add_on_map.reset_index(name='primary_code_array')

C:\Users\hamupratik\AppData\Local\Temp\ipykernel_21772\1804790787.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  add_on_map = referance_data_target_referance.groupby('add_on_code').apply(lambda x: set(x['primary_code']))


In [ ]:
df = df.merge(add_on_map, left_on = ['procedure_code'], right_on = ['add_on_code'], how = 'left')


,payer_control_number,adjudication_record_locator,member_record_locator,member_medicare_id,rendering_provider_npi,service_date,procedure_code,proc_modifier,proc_modifier2,proc_modifier3,...,final_claim_indicator,financial_recovery_indicator,is_dmg_claim,discharge_status_code,is_final_paid_indicator,load_date,root_payer_control_number,query_name,add_on_code,primary_code_array
0,AJX22E9RCH,AJX22E9RCH,DE5JGJ,4C24NH7VN71,1174687495,2025-05-07,99152,GC,NaN,NaN,...,true,false,false,NaN,1,2025-06-02 17:15:00.000,AJX22E9RCH,prof_anylob_aoc_primary_code_unpaid,NaN,NaN
1,AJX22E9RCH,AJX22E9RCH,DE5JGJ,4C24NH7VN71,1174687495,2025-05-07,33208,KX,GC,NaN,...,true,false,false,NaN,1,2025-06-02 17:15:00.000,AJX22E9RCH,prof_anylob_aoc_primary_code_unpaid,NaN,NaN
2,AJX22E9RCH,AJX22E9RCH,DE5JGJ,4C24NH7VN71,1174687495,2025-05-07,33225,GC,NaN,NaN,...,true,false,false,NaN,1,2025-06-02 17:15:00.000,AJX22E9RCH,prof_anylob_aoc_primary_code_unpaid,33225,"{33208, 33223, 33217, 33212, 33229, 33213, 332..."
3,AJX23C7Z5A,AJX23C7Z5A,D968C4,2E36WK7HR84,1053463687,2024-10-22,17004,59,NaN,NaN,...,true,false,false,NaN,1,2024-12-02 17:15:00.000,AJX23C7Z5A,prof_anylob_aoc_primary_code_unpaid,NaN,NaN
4,AJX23C7Z5A,AJX23C7Z5A,D968C4,2E36WK7HR84,1053463687,2024-10-22,11102,NaN,NaN,NaN,...,true,false,false,NaN,1,2024-12-02 17:15:00.000,AJX23C7Z5A,prof_anylob_aoc_primary_code_unpaid,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
757,AJXZH39U49,AJXZH39U49,D6W373,3TQ2RH5NR04,1073745733,2024-12-03,70553,NaN,NaN,NaN,...,true,false,false,NaN,1,2025-04-02 17:12:00.000,AJXZH39U49,prof_anylob_aoc_primary_code_unpaid,NaN,NaN
758,AJXZH39U49,AJXZH39U49,D6W373,3TQ2RH5NR04,1073745733,2024-12-03,0866T,NaN,NaN,NaN,...,true,false,false,NaN,1,2025-04-02 17:12:00.000,AJXZH39U49,prof_anylob_aoc_primary_code_unpaid,0866T,"{70553, 70552, 70551}"
759,AJXZH39U49,AJXZH39U49,D6W373,3TQ2RH5NR04,1073745733,2024-12-03,A9575,NaN,NaN,NaN,...,true,false,false,NaN,1,2025-04-02 17:12:00.000,AJXZH39U49,prof_anylob_aoc_primary_code_unpaid,NaN,NaN
760,AJXZR5CCKR-1,AJXFJ657E2,D2ER62,5NE7JX2JT65,1497701882,2024-10-16,29823,LT,NaN,NaN,...,true,true,false,NaN,1,2025-02-02 17:00:00.000,AJXZR5CCKR,prof_anylob_aoc_primary_code_unpaid,NaN,NaN


In [40]:
df['is_add_on'] =  df.apply(lambda x: '1' if x['plan_paid_amount']>=75 and x['add_on_code'] == x['add_on_code']
                                     else 0, axis=1)

In [44]:
map_add_on_primary_1 = df[(df['is_add_on'] == '1') & (df['plan_paid_amount']>75)][['payer_control_number', 
                                                             'add_on_code', 'primary_code_array', 'plan_paid_amount']]

map_add_on_primary_11 = map_add_on_primary_1.groupby(['payer_control_number'])['primary_code_array'].apply(lambda x: x.tolist()[0]).reset_index()

map_add_on_primary_11['primary_code'] = map_add_on_primary_11['primary_code_array']
map_add_on_primary_11.drop(columns=['primary_code_array'], inplace=True)

df = df.merge(map_add_on_primary_11, how='left', on = 'payer_control_number')


In [46]:
df['is_primary'] = df.apply(
    lambda row: '1' if row['procedure_code'] in row['primary_code'] else '0',
    axis=1
)

In [47]:
df['service_date'] = pd.to_datetime(df['service_date'])
df['proc_code_flag'] = 'other'

for idx, row in df[df['is_add_on'] == '1'].iterrows():
    match = df[
        (df['payer_control_number'] == row['payer_control_number']) &
        (df['service_date'] == row['service_date']) &
        (df['is_primary'] == '1')
    ]
    if not match.empty:
        df.at[idx, 'proc_code_flag'] = 'target'
        for match_idx in match.index:
            df.at[match_idx, 'proc_code_flag'] = 'reference'


In [52]:
valid_pcns = set(df[df['proc_code_flag']=='target']['payer_control_number'].unique())
print('number of pcns pulled from the query in athena:', df['payer_control_number'].nunique())
print("number of pcn dropped because it does not contain any add on code:", df['payer_control_number'].nunique() - len(valid_pcns))
print("%drop:", round((df['payer_control_number'].nunique() - len(valid_pcns))*100 / df['payer_control_number'].nunique(),2))

print("remaining pcn:", len(valid_pcns))

df = df[df['payer_control_number'].isin(valid_pcns)]


number of pcns pulled from the query in athena: 212
number of pcn dropped because it does not contain any add on code: 0
%drop: 0.0
remaining pcn: 212


In [54]:
# There are 129 add_on_codes in the FILTERED REFERANCE TABLE which are also primary codes.
primary_codes_add_on_codes = set(referance_data_target_referance["add_on_code"]).intersection(referance_data_target_referance["primary_code"])
print("number of add_on_codes which are also used as primary code:", len(primary_codes_add_on_codes))

# if add on code is flagged as target but it is present in primary code column in the referance table, 
# we should not select such claim as we dont know if it is billed as add on code or primary code. this code can be primary code by itself

pcns_to_be_dropped = df[(df['proc_code_flag'] == 'target') 
                             & (df['procedure_code'].isin(primary_codes_add_on_codes))]["payer_control_number"].unique()

print("number of pcn to be dropped because of add on code present in primary code:", len(pcns_to_be_dropped))


print("%drop: ", round((len(pcns_to_be_dropped))*100 / df['payer_control_number'].nunique(), 2))

# dropping pcns 
df = df[~df['payer_control_number'].isin(pcns_to_be_dropped)].reset_index(drop = True)


number of add_on_codes which are also used as primary code: 0
number of pcn to be dropped because of add on code present in primary code: 0
%drop:  0.0


In [55]:
pcns_to_be_dropped = df[(df['proc_code_flag'] == 'reference') & (df['plan_paid_amount'] > 0)]['payer_control_number'].unique()

print("number of PCN to be dropped becasue thos rule 3:", len(pcns_to_be_dropped))
print("%drop: ", round((len(pcns_to_be_dropped))*100/ df['payer_control_number'].nunique(), 2))


df = df[~df['payer_control_number'].isin(pcns_to_be_dropped)].reset_index(drop = True)


number of PCN to be dropped becasue thos rule 3: 1
%drop:  0.47


In [56]:
# add above rule
x = referance_data_target_referance.groupby('add_on_code')['primary_code'].unique().reset_index()
x['is_ccccc'] = x['primary_code'].astype(str).apply(lambda x: '1' if "CCCCC" in x else '0')
add_on_codes_to_drop = x[x['is_ccccc'] == '1']['add_on_code'].unique()
print("number of procedure codes with CCCCC as primary code:", len(add_on_codes_to_drop))

# check if this is 0, if yes good to go

number of procedure codes with CCCCC as primary code: 0


In [61]:
# check if each pcn has target reference present or not the claims which are flagged as add_on_combination_type == both_present
for i in set(df["payer_control_number"]):
    z = df[df["payer_control_number"] == i]
    for _, j in z.iterrows():
        try:
            flags = z['proc_code_flag'].unique()
            if ('target' in flags) and ('reference' in flags):
                pass
            else:
                print(j['payer_control_number'])
        except Exception as e:
            print(f"Error processing payer_control_number {j['payer_control_number']}: {e} for rule each pcn has target reference present or not")


In [68]:
df.sort_values(by=['payer_control_number', 'adjudication_record_locator', 'member_record_locator', 'member_medicare_id',
                'rendering_provider_npi', 'service_date'],
                inplace=True)
df.reset_index(drop = True, inplace = True)

In [70]:
# return this
df

,payer_control_number,adjudication_record_locator,member_record_locator,member_medicare_id,rendering_provider_npi,service_date,procedure_code,proc_modifier,proc_modifier2,proc_modifier3,...,is_final_paid_indicator,load_date,root_payer_control_number,query_name,add_on_code,primary_code_array,is_add_on,primary_code,is_primary,proc_code_flag
0,AJX22E9RCH,AJX22E9RCH,DE5JGJ,4C24NH7VN71,1174687495,2025-05-07,99152,GC,NaN,NaN,...,1,2025-06-02 17:15:00.000,AJX22E9RCH,prof_anylob_aoc_primary_code_unpaid,NaN,NaN,0,"{33208, 33223, 33217, 33212, 33229, 33213, 332...",0,other
1,AJX22E9RCH,AJX22E9RCH,DE5JGJ,4C24NH7VN71,1174687495,2025-05-07,33208,KX,GC,NaN,...,1,2025-06-02 17:15:00.000,AJX22E9RCH,prof_anylob_aoc_primary_code_unpaid,NaN,NaN,0,"{33208, 33223, 33217, 33212, 33229, 33213, 332...",1,reference
2,AJX22E9RCH,AJX22E9RCH,DE5JGJ,4C24NH7VN71,1174687495,2025-05-07,33225,GC,NaN,NaN,...,1,2025-06-02 17:15:00.000,AJX22E9RCH,prof_anylob_aoc_primary_code_unpaid,33225,"{33208, 33223, 33217, 33212, 33229, 33213, 332...",1,"{33208, 33223, 33217, 33212, 33229, 33213, 332...",0,target
3,AJX23C7Z5A,AJX23C7Z5A,D968C4,2E36WK7HR84,1053463687,2024-10-22,17004,59,NaN,NaN,...,1,2024-12-02 17:15:00.000,AJX23C7Z5A,prof_anylob_aoc_primary_code_unpaid,NaN,NaN,0,"{11104, 11106, 11102}",0,other
4,AJX23C7Z5A,AJX23C7Z5A,D968C4,2E36WK7HR84,1053463687,2024-10-22,11102,NaN,NaN,NaN,...,1,2024-12-02 17:15:00.000,AJX23C7Z5A,prof_anylob_aoc_primary_code_unpaid,NaN,NaN,0,"{11104, 11106, 11102}",1,reference
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,AJXZH39U49,AJXZH39U49,D6W373,3TQ2RH5NR04,1073745733,2024-12-03,70553,NaN,NaN,NaN,...,1,2025-04-02 17:12:00.000,AJXZH39U49,prof_anylob_aoc_primary_code_unpaid,NaN,NaN,0,"{70553, 70552, 70551}",1,reference
752,AJXZH39U49,AJXZH39U49,D6W373,3TQ2RH5NR04,1073745733,2024-12-03,0866T,NaN,NaN,NaN,...,1,2025-04-02 17:12:00.000,AJXZH39U49,prof_anylob_aoc_primary_code_unpaid,0866T,"{70553, 70552, 70551}",1,"{70553, 70552, 70551}",0,target
753,AJXZH39U49,AJXZH39U49,D6W373,3TQ2RH5NR04,1073745733,2024-12-03,A9575,NaN,NaN,NaN,...,1,2025-04-02 17:12:00.000,AJXZH39U49,prof_anylob_aoc_primary_code_unpaid,NaN,NaN,0,"{70553, 70552, 70551}",0,other
754,AJXZR5CCKR-1,AJXFJ657E2,D2ER62,5NE7JX2JT65,1497701882,2024-10-16,29823,LT,NaN,NaN,...,1,2025-02-02 17:00:00.000,AJXZR5CCKR,prof_anylob_aoc_primary_code_unpaid,NaN,NaN,0,"{29828, 29821, 29806, 29819, 29822, 29824, 298...",1,reference
